<img src="../images/felipepelaquim-9c7CK4BemmA-unsplash.jpg"  style="float: center; margin: 0px; height: 350px; width: 1000px"> 

# Afrobeats influence in Hip Hop

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import time

In [3]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id = '2101cd224f5948e19c4c782d76744ed3',
                                                      client_secret = '879abdfca432449facc9d8566fb40ab6')

Create spotipy object using SpotifyClientCredentials

In [4]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
# results = sp.playlist_tracks('5ZCzd0nCLqiIX1jwQWfazW')
# tracks = results['items']
    
# while results['next']:
#     results = sp.next(results)
#     tracks.extend(results['items'])

# tracks[4]['track'].keys()

# tracks[4]['track']['uri']

## Get song info

Create a function that takes in a playlist of songs and extracts the songs information and features such as artist name, genre, album name, track_uri, danceability, energy, loudness, instrumentalness etc

In [6]:
def get_track_info(playlist):
    
    #split the playlist_uri is at the end of the playlists url. I'll use the .split method to extract it
    uri = playlist.split("/")[-1].split("?")[0]
    
    #from the spotipy library, use the playlist_tracks() method to extract each track from the playlist uri
    #It comes in a nested dictionary format
    
    results = sp.playlist_tracks(uri)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    
    #create an empty dictionary with the info we want to extract as columns
    info = {
    'track_uri':[],
    'track_name':[],
    'artist_name':[],
    'artist_info':[],
    'artist_uri':[],
    'artist_popularity':[],
    'artist_genre':[],
    'album':[],
    'track_pop':[],
    }
    
    features = {'danceability': [],
     'energy': [],
     'key': [],
     'loudness': [],
     'mode': [],
     'speechiness': [],
     'acousticness': [],
     'instrumentalness': [],
     'liveness': [],
     'valence': [],
     'tempo': [],
     'type': [],
     'id': [],
     'uri': [],
     'track_href': [],
     'analysis_url': [],
     'duration_ms': [],
     'time_signature': []
               }
    
    #using a for loop, get the the info for each song and put it into the empty dictionary
    for track in tracks:
        #URI
        info['track_uri'].append((track["track"]["uri"]).split(':')[2])

        #Track name
        info['track_name'].append(track["track"]["name"])

        #Main Artist
        info['artist_uri'].append((track["track"]["artists"][0]["uri"]).split(':')[2])
        info['artist_info'].append(sp.artist(track["track"]["artists"][0]["uri"]))

        #Name, popularity, genre
        info['artist_name'].append(track["track"]["artists"][0]["name"])
        info['artist_popularity'].append(sp.artist(track["track"]["artists"][0]["uri"])["popularity"])
        info['artist_genre'].append(sp.artist(track["track"]["artists"][0]["uri"])["genres"])

        #Album
        info['album'].append(track["track"]["album"]["name"])

        #Popularity of the track
        info['track_pop'].append(track["track"]["popularity"])
        
        #Transform the info dictionary into a dataframe
        info_df = pd.DataFrame(info)
        
        #loop through the tracks to their features and assign it to the empty dictionary
        track_uri = track["track"]["uri"].split(':')[2] 
        
        try:
            for key,value in (sp.audio_features(track_uri)[0]).items():
                features[key].append(value)
            
        except:
            print(f'failed on track {track["track"]["name"]}')
            continue
        #time.sleep(1)
        
    #Transform the features dictionary into a dataframe
    features_df = pd.DataFrame(features)
    
    #drop features we do not need
    #features_df.drop(columns=['type','id','uri','track_href','analysis_url','duration_ms','time_signature'], inplace=True)   
    
    
    
     
    
    return info_df.join(features_df)
        

#### Create dataframe of users playlist

#### Afrobeats playlist

In [7]:
def raw_data(user_playlist_url, genre):
    user_playlist_info = get_track_info(user_playlist_url)
    #add user genre
    user_playlist_info.loc[:,'genre'] = genre
    return user_playlist_info

In [18]:
afrobeats_playlist_url= "https://open.spotify.com/playlist/5ZCzd0nCLqiIX1jwQWfazW"

In [19]:
afrobeats_df = raw_data(afrobeats_playlist_url, 'afrobeats')

failed on track Lonely


In [20]:
afrobeats_df.head(2)

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0mDRuZmReEm6DquPLJlcEm,Oleku (feat. Brymo),Ice Prince,{'external_urls': {'spotify': 'https://open.sp...,1sSt1DqqqFLkPwfrqafVyn,54,"[afro dancehall, afropop, azontobeats, nigeria...",Oleku (feat. Brymo),51,0.544,...,0.751,162.112,audio_features,0mDRuZmReEm6DquPLJlcEm,spotify:track:0mDRuZmReEm6DquPLJlcEm,https://api.spotify.com/v1/tracks/0mDRuZmReEm6...,https://api.spotify.com/v1/audio-analysis/0mDR...,291364.0,5.0,afrobeats
1,1tvi8tv0eykhNcV1WtaIqO,Move Back,5five,{'external_urls': {'spotify': 'https://open.sp...,37zb1JQnDV9dRLatrASEj1,29,"[afro dancehall, azonto, hiplife]",Move Back,45,0.793,...,0.793,125.032,audio_features,1tvi8tv0eykhNcV1WtaIqO,spotify:track:1tvi8tv0eykhNcV1WtaIqO,https://api.spotify.com/v1/tracks/1tvi8tv0eykh...,https://api.spotify.com/v1/audio-analysis/1tvi...,237107.0,4.0,afrobeats


In [21]:
len(afrobeats_df)

1875

In [22]:
afrobeats_df.isna().sum().sum()

18

In [23]:
afrobeats_df[afrobeats_df['energy'].isnull()]

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
1874,4bQga8PJWPDIQmecRrqxj9,N'y pense plus,Tayc,{'external_urls': {'spotify': 'https://open.sp...,7gU9VyFRN3JWPJ5oHOil60,75,[pop urbaine],Fleur froide - Second état : la cristallisation,48,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,afrobeats


In [24]:
afrobeats_df.dropna(inplace=True)

In [25]:
afrobeats_df.isnull().sum().sum()

0

#### Exporta afrobeats as a csv

In [26]:
afrobeats_df.to_csv('../data/afrobeats.csv', index = False)

#### Jacks playlist

In [8]:
jacks_playlist_url = 'https://open.spotify.com/playlist/6UlskZAcTPzcGMnQaMnIVm?si=5cbb031d1fdd4064'

In [9]:
jacks_playlist_df = raw_data(jacks_playlist_url, 'jack')

In [10]:
jacks_playlist_df.head()

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,1YzcrcgR3T2RwAZg5tSvYP,Die Walküre / Erster Aufzug: Orchestervorspiel,Richard Wagner,{'external_urls': {'spotify': 'https://open.sp...,1C1x4MVkql8AiABuTw6DgE,58,"[classical, german opera, german romanticism, ...",Solti - Wagner - The Operas,18,0.2650,...,0.1640,113.033,audio_features,1YzcrcgR3T2RwAZg5tSvYP,spotify:track:1YzcrcgR3T2RwAZg5tSvYP,https://api.spotify.com/v1/tracks/1YzcrcgR3T2R...,https://api.spotify.com/v1/audio-analysis/1Yzc...,196000,3,jack
1,6JmduA0I9QYtD1RiHQgWjj,"Götterdämmerung, WWV 86D, Act III: Siegfrieds ...",Richard Wagner,{'external_urls': {'spotify': 'https://open.sp...,1C1x4MVkql8AiABuTw6DgE,58,"[classical, german opera, german romanticism, ...","Wagner: Götterdämmerung, WWV 86D",5,0.1170,...,0.0396,66.858,audio_features,6JmduA0I9QYtD1RiHQgWjj,spotify:track:6JmduA0I9QYtD1RiHQgWjj,https://api.spotify.com/v1/tracks/6JmduA0I9QYt...,https://api.spotify.com/v1/audio-analysis/6Jmd...,409787,3,jack
2,1U1i1HBJ5H8DY5J4fO8ySg,Tannhäuser: Overture,Richard Wagner,{'external_urls': {'spotify': 'https://open.sp...,1C1x4MVkql8AiABuTw6DgE,58,"[classical, german opera, german romanticism, ...",Wagner: Orchestral Favourites,47,0.0897,...,0.0579,81.802,audio_features,1U1i1HBJ5H8DY5J4fO8ySg,spotify:track:1U1i1HBJ5H8DY5J4fO8ySg,https://api.spotify.com/v1/tracks/1U1i1HBJ5H8D...,https://api.spotify.com/v1/audio-analysis/1U1i...,853827,4,jack
3,5loYnrcJ1XTIbs0MXKntlr,"Götterdämmerung, WWV 86D, Prologue: Siegfrieds...",Richard Wagner,{'external_urls': {'spotify': 'https://open.sp...,1C1x4MVkql8AiABuTw6DgE,58,"[classical, german opera, german romanticism, ...","Wagner: Götterdämmerung, WWV 86D",1,0.2270,...,0.0622,128.729,audio_features,5loYnrcJ1XTIbs0MXKntlr,spotify:track:5loYnrcJ1XTIbs0MXKntlr,https://api.spotify.com/v1/tracks/5loYnrcJ1XTI...,https://api.spotify.com/v1/audio-analysis/5loY...,309053,3,jack
4,3ci6KfIfr3UiIRgJ3WVYhd,"Piano Concerto No. 1 in D Minor, Op. 15: I. Ma...",Johannes Brahms,{'external_urls': {'spotify': 'https://open.sp...,5wTAi7QkpP6kp8a54lmTOq,68,"[classical, german romanticism, late romantic ...",Brahms: Piano Concerto No.1,34,0.1870,...,0.0395,87.459,audio_features,3ci6KfIfr3UiIRgJ3WVYhd,spotify:track:3ci6KfIfr3UiIRgJ3WVYhd,https://api.spotify.com/v1/tracks/3ci6KfIfr3Ui...,https://api.spotify.com/v1/audio-analysis/3ci6...,1407000,4,jack


#### Exporta Jack's playlist as a csv

In [12]:
jacks_playlist_df.to_csv('../data/jack.csv', index = False)

#### Ankita playlist

In [13]:
ankita_playlist_url = 'https://open.spotify.com/playlist/6qzbkhrmxdFj5TtaXR0sfI?si=ElY40mMjQ7apOsHTGlQI7A'

In [14]:
ankita_playlist_df = raw_data(ankita_playlist_url, 'ankita')

In [15]:
ankita_playlist_df.head()

,track_uri,track_name,artist_name,artist_info,artist_uri,artist_popularity,artist_genre,album,track_pop,danceability,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,068HSvCf5MbQfhV4qqaelg,"Haan Main Galat (From ""Love Aaj Kal"")",Pritam,{'external_urls': {'spotify': 'https://open.sp...,1wRPtKGflJrBx9BmLsSwlU,84,"[desi pop, filmi, indian instrumental, modern ...","Haan Main Galat (From ""Love Aaj Kal"")",64,0.685,...,0.893,117.979,audio_features,068HSvCf5MbQfhV4qqaelg,spotify:track:068HSvCf5MbQfhV4qqaelg,https://api.spotify.com/v1/tracks/068HSvCf5MbQ...,https://api.spotify.com/v1/audio-analysis/068H...,218644,4,ankita
1,2aHz87L2Z4a0ZEQ7vMlH8z,Badal Pe Paon Hai,Salim–Sulaiman,{'external_urls': {'spotify': 'https://open.sp...,6ohaQzKaXrobAL8paLSaxq,64,"[desi pop, filmi, indian folk, modern bollywood]",Chak De India,57,0.649,...,0.869,104.999,audio_features,2aHz87L2Z4a0ZEQ7vMlH8z,spotify:track:2aHz87L2Z4a0ZEQ7vMlH8z,https://api.spotify.com/v1/tracks/2aHz87L2Z4a0...,https://api.spotify.com/v1/audio-analysis/2aHz...,243931,4,ankita
2,0mACGvgvCcwjfJCPFLeabh,Ziddi Dil,Vishal Dadlani,{'external_urls': {'spotify': 'https://open.sp...,6CXEwIaXYfVJ84biCxqc9k,72,"[desi hip hop, desi pop, filmi, modern bollywood]",Mary Kom,57,0.666,...,0.541,99.007,audio_features,0mACGvgvCcwjfJCPFLeabh,spotify:track:0mACGvgvCcwjfJCPFLeabh,https://api.spotify.com/v1/tracks/0mACGvgvCcwj...,https://api.spotify.com/v1/audio-analysis/0mAC...,286093,4,ankita
3,0dT6g7oqjEqphjXMk8MKHz,Dangal,Pritam,{'external_urls': {'spotify': 'https://open.sp...,1wRPtKGflJrBx9BmLsSwlU,84,"[desi pop, filmi, indian instrumental, modern ...",Dangal (Original Motion Picture Soundtrack),59,0.618,...,0.665,178.017,audio_features,0dT6g7oqjEqphjXMk8MKHz,spotify:track:0dT6g7oqjEqphjXMk8MKHz,https://api.spotify.com/v1/tracks/0dT6g7oqjEqp...,https://api.spotify.com/v1/audio-analysis/0dT6...,299326,4,ankita
4,77UjLW8j5UAGAGVGhR5oUK,Pray For Me (with Kendrick Lamar),The Weeknd,{'external_urls': {'spotify': 'https://open.sp...,1Xyo4u8uXC1ZmMpatF05PJ,97,"[canadian contemporary r&b, canadian pop, pop]",Black Panther The Album Music From And Inspire...,75,0.735,...,0.188,100.584,audio_features,77UjLW8j5UAGAGVGhR5oUK,spotify:track:77UjLW8j5UAGAGVGhR5oUK,https://api.spotify.com/v1/tracks/77UjLW8j5UAG...,https://api.spotify.com/v1/audio-analysis/77Uj...,211440,4,ankita


#### Export Ankita's playlist as a csv

In [16]:
ankita_playlist_df.to_csv('../data/ankita.csv', index = False)

### Get top Afrobeats songs from 2011 to 2022

In [28]:
afrobeats_url_2011 = 'https://open.spotify.com/playlist/7IFO0RwgGq1GTZYH52FtJK'

In [29]:
afrobeats_url_2012 = 'https://open.spotify.com/playlist/0IsKxqhBrseS4cjSvCZI7I'

In [30]:
afrobeats_url_2013 = 'https://open.spotify.com/playlist/2NXEKf7dKn0NNQCuj4Rniz'

In [31]:
afrobeats_url_2014 = 'https://open.spotify.com/playlist/6Vy0hXleg1ojR4mq57w4Fg'

In [32]:
afrobeats_url_2015 = 'https://open.spotify.com/playlist/2fJOQI1m08OKVvjqZyJjCJ'

In [33]:
afrobeats_url_2016 = 'https://open.spotify.com/playlist/5D8Ri2l5XSAsuzZrRVvFqR'

In [34]:
afrobeats_url_2017 = 'https://open.spotify.com/playlist/1yEJSlGQtlpJKTKYEtIed9'

In [35]:
afrobeats_url_2018 = 'https://open.spotify.com/playlist/4KOcyBABIVdTsTGQCK83CP'

In [36]:
afrobeats_url_2019 = 'https://open.spotify.com/playlist/0n79FLMnGldYuQ4yHymtlg'

In [37]:
afrobeats_url_2020 = 'https://open.spotify.com/playlist/7FEPeJeFl364hbm94hUWbo'

In [38]:
afrobeats_url_2021 = 'https://open.spotify.com/playlist/2WiB3m5N21XCO4CP3eKrLP'

In [39]:
afrobeats_url_2022 = 'https://open.spotify.com/playlist/0aUt6gxNcV6B2La64Bm4D5'

#### Function to get create a dataframe of all the songs from 2011 to 2012

In [40]:
afrobeats_2011 = get_track_info(afrobeats_url_2011)

In [41]:
year_list = [afrobeats_url_2011, afrobeats_url_2012, afrobeats_url_2013, afrobeats_url_2014, afrobeats_url_2015, afrobeats_url_2016,
            afrobeats_url_2017,afrobeats_url_2018]

In [ ]:
for playlist in year_list:
    